<a href="https://colab.research.google.com/github/RajeshworM/fasal_district_panel_regression/blob/main/fasal_district.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Step 1: Install necessary packages
!pip install linearmodels

In [1]:
# Step 2: Mount Google Drive (if using Google Drive for data storage)
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd

In [6]:
# Step 3: Set the file path (replace with your file path)
file_path = '/content/drive/My Drive/panel_data_2000_2024.csv'
data = pd.read_csv(file_path)
data.head()
data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1675 entries, 0 to 1674
Data columns (total 54 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   state              1675 non-null   object 
 1   year               1675 non-null   int64  
 2   district           1675 non-null   object 
 3   stateid            1675 non-null   int64  
 4   districtid         1675 non-null   int64  
 5   area               1514 non-null   float64
 6   yield              1514 non-null   float64
 7   prod               1514 non-null   float64
 8   irrigation_nia     1654 non-null   float64
 9   irrigation_canal   1654 non-null   float64
 10  irrigation_well    1654 non-null   float64
 11  irrigation_tank    1652 non-null   float64
 12  irrigation_others  1650 non-null   float64
 13  soil_type          1675 non-null   int64  
 14  fert1              1675 non-null   float64
 15  fert2              1675 non-null   float64
 16  fert3              1675 

In [ ]:
# Step 4: Load the data (including independent variables for 2021 to 2024)
import pandas as pd
from linearmodels.panel import RandomEffects



In [ ]:
# Step 5: Set the panel index (stateid, districtid, year)
data = data.set_index(['stateid', 'districtid', 'year'])

KeyError: "None of ['stateid', 'districtid', 'year'] are in the columns"

In [ ]:
# Filter the data for the sample period (2000 to 2020)
data_sample = data.loc[data.index.get_level_values('year') <= 2020]

In [ ]:
# Step 6: Calculate the 1-year lag for the average price using .loc
data_sample['avg_price'] = data_sample[['price_crop10m11', 'price_crop11m11', 'price_crop12m11']].mean(axis=1)

# Apply the shift with .loc to modify the DataFrame
data_sample.loc[:, 'avg_price_lag1'] = data_sample.groupby(['stateid', 'districtid'])['avg_price'].shift(1)


<ipython-input-65-e98f3c68a5e4>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_sample['avg_price'] = data_sample[['price_crop10m11', 'price_crop11m11', 'price_crop12m11']].mean(axis=1)
<ipython-input-65-e98f3c68a5e4>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_sample.loc[:, 'avg_price_lag1'] = data_sample.groupby(['stateid', 'districtid'])['avg_price'].shift(1)


In [ ]:
# Step 7: Generate variables by dividing msp11 and the lagged avg_price by fert1, fert2, and fert3
data_sample['msp11_fert1'] = data_sample['msp11'] / data_sample['fert1']
data_sample['msp11_fert2'] = data_sample['msp11'] / data_sample['fert2']
data_sample['msp11_fert3'] = data_sample['msp11'] / data_sample['fert3']

data_sample['avg_price_fert1'] = data_sample['avg_price_lag1'] / data_sample['fert1']
data_sample['avg_price_fert2'] = data_sample['avg_price_lag1'] / data_sample['fert2']
data_sample['avg_price_fert3'] = data_sample['avg_price_lag1'] / data_sample['fert3']

<ipython-input-66-ecdf530dd22d>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_sample['msp11_fert1'] = data_sample['msp11'] / data_sample['fert1']
<ipython-input-66-ecdf530dd22d>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_sample['msp11_fert2'] = data_sample['msp11'] / data_sample['fert2']
<ipython-input-66-ecdf530dd22d>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in t

In [ ]:
# Step 8: Use rainfall variables month-wise (May to December)
rainfall_vars = ['rainfall5', 'rainfall6', 'rainfall7', 'rainfall8', 'rainfall9', 'rainfall10', 'rainfall11', 'rainfall12']

In [ ]:
# Step 9: Use reservoir variables
reservoir_vars = ['res_1', 'res_2', 'res_3', 'res_4', 'res_5', 'res_6', 'res_7', 'res_8', 'res_9']

In [ ]:
# Step 10: Use fortnightly temperature data from July to October
temperature_vars = ['july1_max', 'aug1_max', 'sep1_max', 'oct1_max', 'aug2_max', 'sep2_max', 'oct2_max',
                    'july1_min', 'aug1_min', 'sep1_min', 'oct1_min', 'july2_min', 'aug2_min', 'sep2_min', 'oct2_min']

In [ ]:
# Step 11: Convert soil_type into dummy variables
soil_dummies = pd.get_dummies(data_sample['soil_type'], prefix='soil', drop_first=True)
data_sample = pd.concat([data_sample, soil_dummies], axis=1)

In [ ]:
# Step 12: Generate interaction terms for the different irrigation types
for irrigation_type in ['irrigation_nia', 'irrigation_canal', 'irrigation_well', 'irrigation_tank', 'irrigation_others']:
    for rainfall_var in rainfall_vars:
        data_sample[f'{rainfall_var}_{irrigation_type}'] = data_sample[rainfall_var] * data_sample[irrigation_type]
    for res_var in reservoir_vars:
        data_sample[f'{res_var}_{irrigation_type}'] = data_sample[res_var] * data_sample[irrigation_type]
    for rainfall_var in rainfall_vars:
        for res_var in reservoir_vars:
            data_sample[f'{rainfall_var}_{res_var}_{irrigation_type}'] = data_sample[rainfall_var] * data_sample[res_var]
    data_sample[f'groundwater_{irrigation_type}'] = data_sample['groundwater'] * data_sample[irrigation_type]
    data_sample[f'groundwater_rainfall_{irrigation_type}'] = data_sample['groundwater'] * data_sample[rainfall_var]

<ipython-input-71-8c6aa7ff69a4>:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data_sample[f'{rainfall_var}_{irrigation_type}'] = data_sample[rainfall_var] * data_sample[irrigation_type]
<ipython-input-71-8c6aa7ff69a4>:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data_sample[f'{rainfall_var}_{irrigation_type}'] = data_sample[rainfall_var] * data_sample[irrigation_type]
<ipython-input-71-8c6aa7ff69a4>:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, whi

In [ ]:
# Step 13: Define the initial set of independent variables
X_full = data_sample[['msp11_fert1', 'msp11_fert2', 'msp11_fert3', 'avg_price_fert1', 'avg_price_fert2', 'avg_price_fert3',
                      'groundwater'] + rainfall_vars + reservoir_vars + temperature_vars]

In [ ]:
# Add all dummy variables for soil types (excluding one to avoid multicollinearity)
X_full = pd.concat([X_full, soil_dummies], axis=1)

# Add all irrigation interaction terms to the model
for irrigation_type in ['irrigation_nia', 'irrigation_canal', 'irrigation_well', 'irrigation_tank', 'irrigation_others']:
    for rainfall_var in rainfall_vars:
        X_full = pd.concat([X_full, data_sample[[f'{rainfall_var}_{irrigation_type}']]], axis=1)
    for res_var in reservoir_vars:
        X_full = pd.concat([X_full, data_sample[[f'{res_var}_{irrigation_type}']]], axis=1)
    for rainfall_var in rainfall_vars:
        for res_var in reservoir_vars:
            X_full = pd.concat([X_full, data_sample[[f'{rainfall_var}_{res_var}_{irrigation_type}']]], axis=1)

y = data_sample['yield']


In [ ]:
print(data_sample.columns)


Index(['state', 'district', 'area', 'yield', 'prod', 'irrigation_nia',
       'irrigation_canal', 'irrigation_well', 'irrigation_tank',
       'irrigation_others',
       ...
       'rainfall12_res_2_irrigation_others',
       'rainfall12_res_3_irrigation_others',
       'rainfall12_res_4_irrigation_others',
       'rainfall12_res_5_irrigation_others',
       'rainfall12_res_6_irrigation_others',
       'rainfall12_res_7_irrigation_others',
       'rainfall12_res_8_irrigation_others',
       'rainfall12_res_9_irrigation_others', 'groundwater_irrigation_others',
       'groundwater_rainfall_irrigation_others'],
      dtype='object', length=518)


In [ ]:
# Assuming 'stateid', 'districtid', and 'year' are your identifiers for panel data
data_sample.set_index(['stateid', 'districtid', 'year'], inplace=True)

# Ensure the independent (X) and dependent (y) variables are also MultiIndexed
X_full = data_sample[independent_vars]  # independent_vars is the list of your independent variables
y = data_sample['yield']  # yield is your dependent variable

# Check the first few rows of the DataFrame
print(X_full.head())
print(X_full.index)



KeyError: "None of ['stateid', 'districtid', 'year'] are in the columns"

In [ ]:
# Step 14: Run Random Effects Model
re_model = RandomEffects(y, X_full)
re_results = re_model.fit()

ValueError: DataFrame input must have a MultiIndex with 2 levels

In [ ]:
# Step 1: Install necessary packages
!pip install linearmodels

# Step 2: Mount Google Drive (if using Google Drive for data storage)
from google.colab import drive
drive.mount('/content/drive')

# Step 3: Set the file path (replace with your file path)
file_path = '/content/drive/My Drive/panel_data_2000_2024.csv'
data = pd.read_csv(file_path)
data.head()

# Step 4: Load the data (including independent variables for 2021 to 2024)
import pandas as pd
from linearmodels.panel import RandomEffects

data = pd.read_csv(file_path)

# Step 5: Set the panel index (stateid, districtid, year)
data = data.set_index(['stateid', 'districtid', 'year'])

# Filter the data for the sample period (2000 to 2020)
data_sample = data.loc[data.index.get_level_values('year') <= 2020]

# Step 6: Calculate the 1-year lag for the average price
data_sample['avg_price_lag1'] = data_sample[['price_crop10m11', 'price_crop11m11', 'price_crop12m11']].mean(axis=1).groupby(level=['stateid', 'districtid']).shift(1)

# Step 7: Generate variables by dividing msp11 and the lagged avg_price by fert1, fert2, and fert3
data_sample['msp11_fert1'] = data_sample['msp11'] / data_sample['fert1']
data_sample['msp11_fert2'] = data_sample['msp11'] / data_sample['fert2']
data_sample['msp11_fert3'] = data_sample['msp11'] / data_sample['fert3']

data_sample['avg_price_fert1'] = data_sample['avg_price_lag1'] / data_sample['fert1']
data_sample['avg_price_fert2'] = data_sample['avg_price_lag1'] / data_sample['fert2']
data_sample['avg_price_fert3'] = data_sample['avg_price_lag1'] / data_sample['fert3']

# Step 8: Use rainfall variables month-wise (May to December)
rainfall_vars = ['rainfall5', 'rainfall6', 'rainfall7', 'rainfall8', 'rainfall9', 'rainfall10', 'rainfall11', 'rainfall12']

# Step 9: Use reservoir variables
reservoir_vars = ['res_1', 'res_2', 'res_3', 'res_4', 'res_5', 'res_6', 'res_7', 'res_8', 'res_9']

# Step 10: Use fortnightly temperature data from July to October
temperature_vars = ['july1_max', 'aug1_max', 'sep1_max', 'oct1_max', 'aug2_max', 'sep2_max', 'oct2_max',
                    'july1_min', 'aug1_min', 'sep1_min', 'oct1_min', 'july2_min', 'aug2_min', 'sep2_min', 'oct2_min']

# Step 11: Convert soil_type into dummy variables
soil_dummies = pd.get_dummies(data_sample['soil_type'], prefix='soil', drop_first=True)
data_sample = pd.concat([data_sample, soil_dummies], axis=1)

# Step 12: Generate interaction terms for the different irrigation types
for irrigation_type in ['irrigation_nia', 'irrigation_canal', 'irrigation_well', 'irrigation_tank', 'irrigation_others']:
    for rainfall_var in rainfall_vars:
        data_sample[f'{rainfall_var}_{irrigation_type}'] = data_sample[rainfall_var] * data_sample[irrigation_type]
    for res_var in reservoir_vars:
        data_sample[f'{res_var}_{irrigation_type}'] = data_sample[res_var] * data_sample[irrigation_type]
    for rainfall_var in rainfall_vars:
        for res_var in reservoir_vars:
            data_sample[f'{rainfall_var}_{res_var}_{irrigation_type}'] = data_sample[rainfall_var] * data_sample[res_var]
    data_sample[f'groundwater_{irrigation_type}'] = data_sample['groundwater'] * data_sample[irrigation_type]
    data_sample[f'groundwater_rainfall_{irrigation_type}'] = data_sample['groundwater'] * data_sample[rainfall_var]

# Step 13: Define the initial set of independent variables
X_full = data_sample[['msp11_fert1', 'msp11_fert2', 'msp11_fert3', 'avg_price_fert1', 'avg_price_fert2', 'avg_price_fert3',
                      'groundwater'] + rainfall_vars + reservoir_vars + temperature_vars]

# Add all dummy variables for soil types (excluding one to avoid multicollinearity)
X_full = pd.concat([X_full, soil_dummies], axis=1)

# Add all irrigation interaction terms to the model
for irrigation_type in ['irrigation_nia', 'irrigation_canal', 'irrigation_well', 'irrigation_tank', 'irrigation_others']:
    for rainfall_var in rainfall_vars:
        X_full = pd.concat([X_full, data_sample[[f'{rainfall_var}_{irrigation_type}']]], axis=1)
    for res_var in reservoir_vars:
        X_full = pd.concat([X_full, data_sample[[f'{res_var}_{irrigation_type}']]], axis=1)
    for rainfall_var in rainfall_vars:
        for res_var in reservoir_vars:
            X_full = pd.concat([X_full, data_sample[[f'{rainfall_var}_{res_var}_{irrigation_type}']]], axis=1)

y = data_sample['yield']

# Step 14: Run Random Effects Model
re_model = RandomEffects(y, X_full)
re_results = re_model.fit()

# Step 15: Select only positively significant variables
significant_vars = re_results.pvalues[re_results.pvalues < 0.05].index
positive_vars = re_results.params[re_results.params > 0].index
significant_positive_vars = list(set(significant_vars).intersection(set(positive_vars)))

print("Significant positive variables:", significant_positive_vars)

# Step 16: Run regression again with only the significant positive variables
X_significant = data_sample[significant_positive_vars]
re_model_significant = RandomEffects(y, X_significant)
re_results_significant = re_model_significant.fit()

print(re_results_significant.summary)

# Step 17: Forecast yield for 2021 to 2024 using the significant positive variables

# Filter the data for the forecast period (2021 to 2024)
forecast_data = data.loc[(data.index.get_level_values('year') >= 2021) & (data.index.get_level_values('year') <= 2024)]

# Use the significant variables for the forecast period
X_forecast = forecast_data[significant_positive_vars]

# Step 18: Predict yields for 2021 to 2024
forecast_data['predicted_yield'] = re_results_significant.predict(X_forecast)

# Step 19: Display the forecasted results
print(forecast_data[['stateid', 'districtid', 'year', 'predicted_yield']])

# Optional: Save forecasted results to a file
forecast_data[['stateid', 'districtid', 'year', 'predicted_yield']].to_csv("/content/drive/My Drive/forecasted_yields_2021_2024.csv", index=False)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


<ipython-input-87-a0f080b5516d>:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_sample['avg_price_lag1'] = data_sample[['price_crop10m11', 'price_crop11m11', 'price_crop12m11']].mean(axis=1).groupby(level=['stateid', 'districtid']).shift(1)
<ipython-input-87-a0f080b5516d>:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_sample['msp11_fert1'] = data_sample['msp11'] / data_sample['fert1']
<ipython-input-87-a0f080b5516d>:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice

ValueError: DataFrame input must have a MultiIndex with 2 levels

In [ ]:
# Step 1: Install necessary packages
!pip install linearmodels statsmodels

# Step 2: Import libraries
import pandas as pd
from linearmodels.panel import RandomEffects
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
import numpy as np

# Step 3: Mount Google Drive (if using Google Drive for data storage)
from google.colab import drive
drive.mount('/content/drive')

# Step 4: Set the file path and load the data
file_path = '/content/drive/My Drive/panel_data_2000_2024.csv'
data = pd.read_csv(file_path)

# Step 5: Declare panel data by creating 'panel_id' as a combination of 'stateid' and 'districtid'
data['panel_id'] = data['stateid'].astype(str) + '_' + data['districtid'].astype(str)
data = data.set_index(['panel_id', 'year'])

# Step 6: Filter the data for the sample period (2000 to 2020)
data_sample = data.loc[data.index.get_level_values('year') <= 2020]

# Step 7: Calculate the 1-year lag for the average price
data_sample['avg_price_lag1'] = data_sample[['price_crop10m11', 'price_crop11m11', 'price_crop12m11']].mean(axis=1).groupby(level='panel_id').shift(1)

# Step 8: Generate variables by dividing msp11 and the lagged avg_price by fert1, fert2, and fert3
data_sample['msp11_fert1'] = data_sample['msp11'] / data_sample['fert1']
data_sample['msp11_fert2'] = data_sample['msp11'] / data_sample['fert2']
data_sample['msp11_fert3'] = data_sample['msp11'] / data_sample['fert3']

data_sample['avg_price_fert1'] = data_sample['avg_price_lag1'] / data_sample['fert1']
data_sample['avg_price_fert2'] = data_sample['avg_price_lag1'] / data_sample['fert2']
data_sample['avg_price_fert3'] = data_sample['avg_price_lag1'] / data_sample['fert3']

# Step 9: Use rainfall variables month-wise (May to December)
rainfall_vars = ['rainfall5', 'rainfall6', 'rainfall7', 'rainfall8', 'rainfall9', 'rainfall10', 'rainfall11', 'rainfall12']

# Step 10: Use reservoir variables
reservoir_vars = ['res_1', 'res_2', 'res_3', 'res_4', 'res_5', 'res_6', 'res_7', 'res_8', 'res_9']

# Step 11: Use fortnightly temperature data from July to October
temperature_vars = ['july1_max', 'aug1_max', 'sep1_max', 'oct1_max', 'aug2_max', 'sep2_max', 'oct2_max',
                    'july1_min', 'aug1_min', 'sep1_min', 'oct1_min', 'july2_min', 'aug2_min', 'sep2_min', 'oct2_min']

# Step 12: Convert soil_type into dummy variables
soil_dummies = pd.get_dummies(data_sample['soil_type'], prefix='soil', drop_first=True)
data_sample = pd.concat([data_sample, soil_dummies], axis=1)

# Step 13: Generate interaction terms for different irrigation types
for irrigation_type in ['irrigation_nia', 'irrigation_canal', 'irrigation_well', 'irrigation_tank', 'irrigation_others']:
    for rainfall_var in rainfall_vars:
        data_sample[f'{rainfall_var}_{irrigation_type}'] = data_sample[rainfall_var] * data_sample[irrigation_type]
    for res_var in reservoir_vars:
        data_sample[f'{res_var}_{irrigation_type}'] = data_sample[res_var] * data_sample[irrigation_type]
    data_sample[f'groundwater_{irrigation_type}'] = data_sample['groundwater'] * data_sample[irrigation_type]

# Step 14: Define the initial set of independent variables
X_full = data_sample[['msp11_fert1', 'msp11_fert2', 'msp11_fert3', 'avg_price_fert1', 'avg_price_fert2', 'avg_price_fert3',
                      'groundwater'] + rainfall_vars + reservoir_vars + temperature_vars]

# Add soil type dummies to X_full
X_full = pd.concat([X_full, soil_dummies], axis=1)

# Step 15: Ensure that X_full contains only numeric data
X_full = X_full.apply(pd.to_numeric, errors='coerce')

# Step 16: Drop rows with missing values in X_full
X_full = X_full.dropna()

# Step 17: Check for multicollinearity by calculating VIF
def calculate_vif(X):
    vif_data = pd.DataFrame()
    vif_data["Variable"] = X.columns
    vif_data["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
    return vif_data

vif_df = calculate_vif(X_full)
print("VIF Data:\n", vif_df)

# Step 18: Drop variables with high VIF (typically VIF > 10) to avoid multicollinearity
high_vif_vars = vif_df[vif_df["VIF"] > 10]["Variable"]
X_full = X_full.drop(columns=high_vif_vars)

# Step 19: Define dependent variable 'y'
y = data_sample['yield']

# Step 20: Run Random Effects Model
re_model = RandomEffects(y, X_full)
re_results = re_model.fit()

print(re_results.summary())

# Step 21: Select only significant variables from price-related section
significant_vars = re_results.pvalues[re_results.pvalues < 0.05].index
positive_vars = re_results.params[re_results.params > 0].index
significant_positive_vars = list(set(significant_vars).intersection(set(positive_vars)))

print("Significant positive variables:", significant_positive_vars)

# Step 22: Run regression again with only the significant positive variables from the price section
X_significant = X_full[significant_positive_vars]
re_model_significant = RandomEffects(y, X_significant)
re_results_significant = re_model_significant.fit()

print(re_results_significant.summary())

# Step 23: Forecast yield for 2021 to 2024 using significant positive variables
forecast_data = data.loc[(data.index.get_level_values('year') >= 2021) & (data.index.get_level_values('year') <= 2024)]

# Use the significant variables for the forecast period
X_forecast = forecast_data[significant_positive_vars]

# Step 24: Predict yields for 2021 to 2024
forecast_data['predicted_yield'] = re_results_significant.predict(X_forecast)

# Step 25: Display forecasted results
print(forecast_data[['panel_id', 'year', 'predicted_yield']])

# Optional: Save forecasted results to a CSV file
forecast_data[['panel_id', 'year', 'predicted_yield']].to_csv("/content/drive/My Drive/forecasted_yields_2021_2024.csv", index=False)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


<ipython-input-93-4abf102772e9>:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_sample['avg_price_lag1'] = data_sample[['price_crop10m11', 'price_crop11m11', 'price_crop12m11']].mean(axis=1).groupby(level='panel_id').shift(1)
<ipython-input-93-4abf102772e9>:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_sample['msp11_fert1'] = data_sample['msp11'] / data_sample['fert1']
<ipython-input-93-4abf102772e9>:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFra

TypeError: ufunc 'isfinite' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''

In [ ]:
# Step 1: Install necessary packages
!pip install linearmodels statsmodels

# Step 2: Import libraries
import pandas as pd
from linearmodels.panel import RandomEffects
from statsmodels.stats.outliers_influence import variance_inflation_factor
import numpy as np

# Step 3: Mount Google Drive (if using Google Drive for data storage)
from google.colab import drive
drive.mount('/content/drive')

# Step 4: Set the file path and load the data
file_path = '/content/drive/My Drive/panel_data_2000_2024.csv'
data = pd.read_csv(file_path)

# Step 5: Declare panel data by creating 'panel_id' as a combination of 'stateid' and 'districtid'
data['panel_id'] = data['stateid'].astype(str) + '_' + data['districtid'].astype(str)
data = data.set_index(['panel_id', 'year'])

# Step 6: Filter the data for the sample period (2000 to 2020)
data_sample = data.loc[data.index.get_level_values('year') <= 2020]

# Step 7: Calculate the 1-year lag for the average price
data_sample['avg_price_lag1'] = data_sample[['price_crop10m11', 'price_crop11m11', 'price_crop12m11']].mean(axis=1).groupby(level='panel_id').shift(1)

# Step 8: Generate variables by dividing msp11 and the lagged avg_price by fert1, fert2, and fert3
data_sample['msp11_fert1'] = data_sample['msp11'] / data_sample['fert1']
data_sample['msp11_fert2'] = data_sample['msp11'] / data_sample['fert2']
data_sample['msp11_fert3'] = data_sample['msp11'] / data_sample['fert3']

data_sample['avg_price_fert1'] = data_sample['avg_price_lag1'] / data_sample['fert1']
data_sample['avg_price_fert2'] = data_sample['avg_price_lag1'] / data_sample['fert2']
data_sample['avg_price_fert3'] = data_sample['avg_price_lag1'] / data_sample['fert3']

# Step 9: Use rainfall variables month-wise (May to December)
rainfall_vars = ['rainfall5', 'rainfall6', 'rainfall7', 'rainfall8', 'rainfall9', 'rainfall10', 'rainfall11', 'rainfall12']

# Step 10: Use reservoir variables
reservoir_vars = ['res_1', 'res_2', 'res_3', 'res_4', 'res_5', 'res_6', 'res_7', 'res_8', 'res_9']

# Step 11: Use fortnightly temperature data from July to October
temperature_vars = ['july1_max', 'aug1_max', 'sep1_max', 'oct1_max', 'aug2_max', 'sep2_max', 'oct2_max',
                    'july1_min', 'aug1_min', 'sep1_min', 'oct1_min', 'july2_min', 'aug2_min', 'sep2_min', 'oct2_min']

# Step 12: Convert soil_type into dummy variables
soil_dummies = pd.get_dummies(data_sample['soil_type'], prefix='soil', drop_first=True)
data_sample = pd.concat([data_sample, soil_dummies], axis=1)

# Step 13: Generate interaction terms for different irrigation types
for irrigation_type in ['irrigation_nia', 'irrigation_canal', 'irrigation_well', 'irrigation_tank', 'irrigation_others']:
    for rainfall_var in rainfall_vars:
        data_sample[f'{rainfall_var}_{irrigation_type}'] = data_sample[rainfall_var] * data_sample[irrigation_type]
    for res_var in reservoir_vars:
        data_sample[f'{res_var}_{irrigation_type}'] = data_sample[res_var] * data_sample[irrigation_type]
    data_sample[f'groundwater_{irrigation_type}'] = data_sample['groundwater'] * data_sample[irrigation_type]

# Step 14: Define the initial set of independent variables
X_full = data_sample[['msp11_fert1', 'msp11_fert2', 'msp11_fert3', 'avg_price_fert1', 'avg_price_fert2', 'avg_price_fert3',
                      'groundwater'] + rainfall_vars + reservoir_vars + temperature_vars]

# Add soil type dummies to X_full
X_full = pd.concat([X_full, soil_dummies], axis=1)

# Step 15: Ensure that X_full contains only numeric data
X_full = X_full.apply(pd.to_numeric, errors='coerce')

# Step 16: Drop rows with missing values in X_full
X_full = X_full.dropna()

# Step 17: Handle infinite values by replacing them with NaN
X_full.replace([np.inf, -np.inf], np.nan, inplace=True)
X_full = X_full.dropna()

# Step 18: Check for multicollinearity by calculating VIF
def calculate_vif(X):
    vif_data = pd.DataFrame()
    vif_data["Variable"] = X.columns
    # Ensure X is numeric and no NaN values exist
    X = X.fillna(0)  # Replace NaN with 0 or another suitable value
    vif_data["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
    return vif_data

vif_df = calculate_vif(X_full)
print("VIF Data:\n", vif_df)

# Step 19: Drop variables with high VIF (typically VIF > 10) to avoid multicollinearity
high_vif_vars = vif_df[vif_df["VIF"] > 10]["Variable"]
X_full = X_full.drop(columns=high_vif_vars)

# Step 20: Define dependent variable 'y'
y = data_sample['yield']

# Step 21: Run Random Effects Model
re_model = RandomEffects(y, X_full)
re_results = re_model.fit()

print(re_results.summary())

# Step 22: Select only significant variables from price-related section
significant_vars = re_results.pvalues[re_results.pvalues < 0.05].index
positive_vars = re_results.params[re_results.params > 0].index
significant_positive_vars = list(set(significant_vars).intersection(set(positive_vars)))

print("Significant positive variables:", significant_positive_vars)

# Step 23: Run regression again with only the significant positive variables from the price section
X_significant = X_full[significant_positive_vars]
re_model_significant = RandomEffects(y, X_significant)
re_results_significant = re_model_significant.fit()

print(re_results_significant.summary())

# Step 24: Forecast yield for 2021 to 2024 using significant positive variables
forecast_data = data.loc[(data.index.get_level_values('year') >= 2021) & (data.index.get_level_values('year') <= 2024)]

# Use the significant variables for the forecast period
X_forecast = forecast_data[significant_positive_vars]

# Step 25: Predict yields for 2021 to 2024
forecast_data['predicted_yield'] = re_results_significant.predict(X_forecast)

# Step 26: Display forecasted results
print(forecast_data[['panel_id', 'year', 'predicted_yield']])

# Optional: Save forecasted results to a CSV file
forecast_data[['panel_id', 'year', 'predicted_yield']].to_csv("/content/drive/My Drive/forecasted_yields_2021_2024.csv", index=False)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 18.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.5/94.5 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 1.3 MB/s eta 0:00:00
Mounted at /content/drive


<ipython-input-1-2fdc3597d742>:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_sample['avg_price_lag1'] = data_sample[['price_crop10m11', 'price_crop11m11', 'price_crop12m11']].mean(axis=1).groupby(level='panel_id').shift(1)
<ipython-input-1-2fdc3597d742>:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_sample['msp11_fert1'] = data_sample['msp11'] / data_sample['fert1']
<ipython-input-1-2fdc3597d742>:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.

TypeError: ufunc 'isfinite' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''

<font size='6'> <font color ='red'> **NEW Program 01/10/2024 for district wise panel regression**

In [9]:
# Step 1: Install necessary packages
!pip install linearmodels statsmodels

# Step 2: Import libraries
import pandas as pd
from linearmodels.panel import RandomEffects
from statsmodels.stats.outliers_influence import variance_inflation_factor
import numpy as np

# Step 3: Mount Google Drive (if using Google Drive for data storage)
from google.colab import drive
drive.mount('/content/drive')

# Step 4: Load the data
file_path = '/content/drive/My Drive/panel_data_2000_2024.csv'  # Adjust the path accordingly
data = pd.read_csv(file_path)

# Step 5: Prepare panel data
data['panel_id'] = data['stateid'].astype(str) + '_' + data['districtid'].astype(str)
data.set_index(['panel_id', 'year'], inplace=True)

# Step 6: Filter the data for the sample period (2000 to 2020)
data_sample = data.loc[data.index.get_level_values('year') <= 2020]

# Step 7: Calculate 1-year lag for the average price
data_sample['avg_price_lag1'] = data_sample[['price_crop10m11', 'price_crop11m11', 'price_crop12m11']].mean(axis=1).groupby(level='panel_id').shift(1)

# Step 8: Generate variables dividing msp11 and the lagged avg_price by fert1, fert2, and fert3
for fert in ['fert1', 'fert2', 'fert3']:
    data_sample[f'msp11_{fert}'] = data_sample['msp11'] / data_sample[fert]
    data_sample[f'avg_price_{fert}'] = data_sample['avg_price_lag1'] / data_sample[fert]

# Step 9: Prepare variables for analysis
rainfall_vars = ['rainfall5', 'rainfall6', 'rainfall7', 'rainfall8', 'rainfall9', 'rainfall10', 'rainfall11', 'rainfall12']
reservoir_vars = ['res_1', 'res_2', 'res_3', 'res_4', 'res_5', 'res_6', 'res_7', 'res_8', 'res_9']
temperature_vars = ['july1_max', 'aug1_max', 'sep1_max', 'oct1_max', 'aug2_max', 'sep2_max', 'oct2_max', 'july1_min', 'aug1_min', 'sep1_min', 'oct1_min', 'july2_min', 'aug2_min', 'sep2_min', 'oct2_min']

# Step 10: Convert soil_type into dummy variables
soil_dummies = pd.get_dummies(data_sample['soil_type'], prefix='soil', drop_first=True)
data_sample = pd.concat([data_sample, soil_dummies], axis=1)

# Step 11: Generate interaction terms
for irrigation_type in ['irrigation_nia', 'irrigation_canal', 'irrigation_well', 'irrigation_tank', 'irrigation_others']:
    for var in rainfall_vars + reservoir_vars:
        data_sample[f'{var}_{irrigation_type}'] = data_sample[var] * data_sample[irrigation_type]
    data_sample[f'groundwater_{irrigation_type}'] = data_sample['groundwater'] * data_sample[irrigation_type]

# Step 12: Define independent variables
X_full = data_sample[['msp11_fert1', 'msp11_fert2', 'msp11_fert3', 'avg_price_fert1', 'avg_price_fert2', 'avg_price_fert3', 'groundwater'] + rainfall_vars + reservoir_vars + temperature_vars]
X_full = pd.concat([X_full, soil_dummies], axis=1)

# Step 13: Ensure X_full contains only numeric data
X_full = X_full.apply(pd.to_numeric, errors='coerce')  # Convert to numeric, setting errors to NaN

# Step 14: Drop rows with NaN values
X_full.dropna(inplace=True)  # Drop rows with NaN values

# Step 15: Check the contents of X_full
print("Contents of X_full:\n", X_full.head())

# Step 16: Define dependent variable 'y'
y = data_sample['yield'].loc[X_full.index]  # Ensure y aligns with the index of X_full

# Step 17: Run Random Effects Model
re_model = RandomEffects(y, X_full)
re_results = re_model.fit()

print(re_results.summary)  # Accessing the summary attribute instead of calling it

# Step 18: Select significant positive variables
significant_vars = re_results.pvalues[re_results.pvalues < 0.05].index
positive_vars = re_results.params[re_results.params > 0].index
significant_positive_vars = list(set(significant_vars).intersection(set(positive_vars)))

print("Significant positive variables:", significant_positive_vars)

# Step 19: Create a new DataFrame for significant variables
X_significant = X_full[significant_positive_vars]

# Step 20: Check for multicollinearity by calculating VIF for significant variables
def calculate_vif(X):
    vif_data = pd.DataFrame()
    vif_data["Variable"] = X.columns
    vif_data["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
    return vif_data

# Step 21: Calculate VIF
vif_df = calculate_vif(X_significant)
print("VIF Data:\n", vif_df)

# Step 22: Drop variables with high VIF (typically VIF > 10)
high_vif_vars = vif_df[vif_df["VIF"] > 10]["Variable"]
X_significant = X_significant.drop(columns=high_vif_vars)

# Step 23: Run regression again with only significant positive variables
re_model_significant = RandomEffects(y, X_significant)
re_results_significant = re_model_significant.fit()

print(re_results_significant.summary)  # Accessing the summary attribute instead of calling it

# Step 24: Forecast yield for 2021 to 2024 using significant positive variables
# Filter data for the forecast period
forecast_data = data.loc[(data.index.get_level_values('year') >= 2021) & (data.index.get_level_values('year') <= 2024)]

# Ensure to reset the index to keep 'panel_id' and 'year' as columns
forecast_data.reset_index(inplace=True)

# Create the MultiIndex for forecasting
forecast_data.set_index(['panel_id', 'year'], inplace=True)

# Use only significant positive variables for the forecast
X_forecast = forecast_data[significant_positive_vars]

# Make sure to include all necessary columns for prediction
# We need to align the exogenous variables with the fitted model's parameters
X_forecast = X_forecast.reindex(columns=significant_positive_vars, fill_value=0)

# Step 25: Predict yields for 2021 to 2024
forecast_data['predicted_yield'] = re_results_significant.predict(X_forecast)

# Step 26: Display forecasted results
# Reset the index to access 'panel_id' and 'year' as columns
forecast_data_reset = forecast_data.reset_index()

# Now print the desired columns
print(forecast_data_reset[['panel_id', 'year', 'predicted_yield']])

# Optional: Save forecasted results to a CSV file
forecast_data_reset[['panel_id', 'year', 'predicted_yield']].to_csv("/content/drive/My Drive/forecasted_yields_2021_2024.csv", index=False)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Contents of X_full:
                msp11_fert1  msp11_fert2  msp11_fert3  avg_price_fert1  \
panel_id year                                                           
7_1      2001     7.443820     6.184364     6.091954        21.900749   
         2002     7.402423     6.373117     6.321839        21.453567   
         2003     7.352941     6.447831     6.395349        21.176471   
         2004     7.427056     6.459054     6.363636        21.237843   
         2005     7.392996     6.404494     6.333333        20.925205   

               avg_price_fert2  avg_price_fert3  groundwater  rainfall5  \
panel_id year                                                             
7_1      2001        18.195255        17.923372        11.64       2.44   
         2002        18.470452        18.321839        10.57       0.00   
         2003        18.569754        

<ipython-input-9-de03be283cf4>:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_sample['avg_price_lag1'] = data_sample[['price_crop10m11', 'price_crop11m11', 'price_crop12m11']].mean(axis=1).groupby(level='panel_id').shift(1)
<ipython-input-9-de03be283cf4>:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_sample[f'msp11_{fert}'] = data_sample['msp11'] / data_sample[fert]
<ipython-input-9-de03be283cf4>:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.


                        RandomEffects Estimation Summary                        
Dep. Variable:                  yield   R-squared:                        0.9023
Estimator:              RandomEffects   R-squared (Between):              0.9710
No. Observations:                 351   R-squared (Within):               0.5902
Date:                Wed, Dec 11 2024   R-squared (Overall):              0.9023
Time:                        06:07:40   Log-likelihood                   -290.03
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      66.142
Entities:                          18   P-value                           0.0000
Avg Obs:                       19.500   Distribution:                  F(43,308)
Min Obs:                       13.000                                           
Max Obs:                       20.000   F-statistic (robust):             66.142
                            

/usr/local/lib/python3.10/dist-packages/linearmodels/panel/model.py:2759: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  super().__init__(dependent, exog, weights=weights, check_rank=check_rank)
<ipython-input-9-de03be283cf4>:119: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  forecast_data['predicted_yield'] = re_results_significant.predict(X_forecast)


In [2]:
# Step 1: Install necessary packages
!pip install linearmodels statsmodels

# Step 2: Import libraries
import pandas as pd
from linearmodels.panel import RandomEffects
import numpy as np

# Step 3: Mount Google Drive (if using Google Drive for data storage)
from google.colab import drive
drive.mount('/content/drive')

# Step 4: Load the data
file_path = '/content/drive/My Drive/panel_data_2000_2024_1.csv'  # Adjust the path accordingly
data = pd.read_csv(file_path)

# Step 5: Prepare panel data
data['panel_id'] = data['stateid'].astype(str) + '_' + data['districtid'].astype(str)
data.set_index(['panel_id', 'year'], inplace=True)

# Step 6: Filter the data for the sample period (2000 to 2020)
data_sample = data.loc[data.index.get_level_values('year') <= 2020]

# Step 7: Calculate 1-year lag for the average price
data_sample['avg_price_lag1'] = data_sample[['price_crop10m11', 'price_crop11m11', 'price_crop12m11']].mean(axis=1).groupby(level='panel_id').shift(1)

# Step 8: Generate variables dividing msp11 and the lagged avg_price by fert1, fert2, and fert3
for fert in ['fert1', 'fert2', 'fert3']:
    data_sample[f'msp11_{fert}'] = data_sample['msp11'] / data_sample[fert]
    data_sample[f'avg_price_{fert}'] = data_sample['avg_price_lag1'] / data_sample[fert]

# Step 9: Prepare variables for analysis
rainfall_vars = ['rainfall5', 'rainfall6', 'rainfall7', 'rainfall8', 'rainfall9', 'rainfall10', 'rainfall11', 'rainfall12']
reservoir_vars = ['res_1', 'res_2', 'res_3', 'res_4', 'res_5', 'res_6', 'res_7', 'res_8', 'res_9']
temperature_vars = ['july1_max', 'aug1_max', 'sep1_max', 'oct1_max', 'aug2_max', 'sep2_max', 'oct2_max', 'july1_min', 'aug1_min', 'sep1_min', 'oct1_min', 'july2_min', 'aug2_min', 'sep2_min', 'oct2_min']

# Step 10: Convert soil_type into dummy variables
soil_dummies = pd.get_dummies(data_sample['soil_type'], prefix='soil', drop_first=True)
data_sample = pd.concat([data_sample, soil_dummies], axis=1)

# Step 11: Generate interaction terms
for irrigation_type in ['irrigation_nia', 'irrigation_canal', 'irrigation_well', 'irrigation_tank', 'irrigation_others']:
    for var in rainfall_vars + reservoir_vars:
        data_sample[f'{var}_{irrigation_type}'] = data_sample[var] * data_sample[irrigation_type]
    data_sample[f'groundwater_{irrigation_type}'] = data_sample['groundwater'] * data_sample[irrigation_type]

# Step 12: Define dependent variable 'y'
y = data_sample['yield'].loc[data_sample.index]

# Step 13: Function to check significance and aggregate variables
def check_significance_and_average(y, data_sample, variables):
    significant_vars = []
    for var in variables:
        # Create a model with the current variable included
        X_temp = data_sample[significant_vars + [var]].dropna()
        y_temp = y.loc[X_temp.index]

        if not y_temp.empty and not X_temp.empty:
            temp_model = RandomEffects(y_temp, X_temp)
            results = temp_model.fit()

            # Check if significant at 5% level
            if results.pvalues[var] < 0.05:
                significant_vars.append(var)
    return significant_vars

# Step 14: Analyze each group of variables
# For price-related variables, only keep positive significant ones
price_vars = ['msp11_fert1', 'msp11_fert2', 'msp11_fert3', 'avg_price_fert1', 'avg_price_fert2', 'avg_price_fert3']
significant_price_vars = check_significance_and_average(y, data_sample, price_vars)

# Calculate average for significant price variables
if significant_price_vars:
    data_sample['average_price'] = data_sample[significant_price_vars].mean(axis=1)
else:
    data_sample['average_price'] = 0

# For groundwater variables
groundwater_vars = ['groundwater']
significant_groundwater_vars = check_significance_and_average(y, data_sample, groundwater_vars)

if significant_groundwater_vars:
    data_sample['average_groundwater'] = data_sample[significant_groundwater_vars].mean(axis=1)
else:
    data_sample['average_groundwater'] = 0

# For reservoir variables, sum them
data_sample['total_reservoir'] = data_sample[reservoir_vars].sum(axis=1)

# For rainfall variables, average them
data_sample['average_rainfall'] = data_sample[rainfall_vars].mean(axis=1)

# For irrigation variables, average them
irrigation_vars = ['irrigation_nia', 'irrigation_canal', 'irrigation_well', 'irrigation_tank', 'irrigation_others']
data_sample['average_irrigation'] = data_sample[irrigation_vars].mean(axis=1)

# For temperature variables, average them
data_sample['average_temperature'] = data_sample[temperature_vars].mean(axis=1)

# Step 15: Create a DataFrame with selected significant variables
selected_vars = ['average_price', 'average_groundwater', 'total_reservoir', 'average_rainfall', 'average_irrigation', 'average_temperature']
X_final = data_sample[selected_vars]

# Import the necessary module for adding a constant
import statsmodels.api as sm

# Step 16: Run the final Random Effects Model with a constant term
# Add a constant to the independent variables
X_final = sm.add_constant(X_final)

# Fit the Random Effects model
re_model_final = RandomEffects(y, X_final)
re_results_final = re_model_final.fit()

# Print final model summary
print(re_results_final.summary)

# Extract and print the constant term
if 'const' in re_results_final.params:
    print("Constant term (Intercept):", re_results_final.params['const'])
else:
    print("Constant term not included in the model.")



# Step 17: Forecast yield for 2021 to 2024 using significant positive variables
# Filter data for the forecast period
forecast_data = data.loc[(data.index.get_level_values('year') >= 2021) & (data.index.get_level_values('year') <= 2024)]

# Ensure to reset the index to keep 'panel_id' and 'year' as columns
forecast_data.reset_index(inplace=True)

# Step 18: Create average and total variables for forecasting
forecast_data['average_price'] = data_sample['average_price'].mean()
forecast_data['average_groundwater'] = data_sample['average_groundwater'].mean()
forecast_data['total_reservoir'] = data_sample['total_reservoir'].mean()
forecast_data['average_rainfall'] = data_sample['average_rainfall'].mean()
forecast_data['average_irrigation'] = data_sample['average_irrigation'].mean()
forecast_data['average_temperature'] = data_sample['average_temperature'].mean()

# Create the MultiIndex for forecasting
forecast_data.set_index(['panel_id', 'year'], inplace=True)

# Prepare final input for prediction
X_forecast = forecast_data[['average_price', 'average_groundwater', 'total_reservoir', 'average_rainfall', 'average_irrigation', 'average_temperature']]
X_forecast = X_forecast.reindex(columns=selected_vars, fill_value=0)

# Step 19: Predict yields for 2021 to 2024
forecast_data['predicted_yield'] = re_results_final.predict(X_forecast)

# Step 20: Display forecasted results
# Reset the index to access 'panel_id' and 'year' as columns
forecast_data_reset = forecast_data.reset_index()

# Now print the desired columns
print(forecast_data_reset[['panel_id', 'year', 'predicted_yield']])

# Optional: Save forecasted results to a CSV file
forecast_data_reset[['panel_id', 'year', 'predicted_yield']].to_csv("/content/drive/My Drive/forecasted_yields_2021_2024.csv", index=False)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


<ipython-input-2-f4279713a185>:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_sample['avg_price_lag1'] = data_sample[['price_crop10m11', 'price_crop11m11', 'price_crop12m11']].mean(axis=1).groupby(level='panel_id').shift(1)
<ipython-input-2-f4279713a185>:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_sample[f'msp11_{fert}'] = data_sample['msp11'] / data_sample[fert]
<ipython-input-2-f4279713a185>:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.


                        RandomEffects Estimation Summary                        
Dep. Variable:                  yield   R-squared:                        0.4598
Estimator:              RandomEffects   R-squared (Between):             -0.1938
No. Observations:                1287   R-squared (Within):               0.4947
Date:                Wed, Dec 11 2024   R-squared (Overall):              0.1829
Time:                        10:02:04   Log-likelihood                   -685.47
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      181.61
Entities:                          62   P-value                           0.0000
Avg Obs:                       20.758   Distribution:                  F(6,1280)
Min Obs:                       13.000                                           
Max Obs:                       21.000   F-statistic (robust):             181.51
                            

<ipython-input-2-f4279713a185>:134: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  forecast_data['average_price'] = data_sample['average_price'].mean()
<ipython-input-2-f4279713a185>:135: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  forecast_data['average_groundwater'] = data_sample['average_groundwater'].mean()
<ipython-input-2-f4279713a185>:136: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the cavea

ValueError: exog does not have the correct number of columns. Saw 6, expected
7. This can happen since exog is converted to a PanelData object
before computing the fitted value. The best practice is to pass a DataFrame
with a 2-level MultiIndex containing the entity- and time-ids.

In [17]:
# Step 1: Install necessary packages
!pip install linearmodels statsmodels

# Step 2: Import libraries
import pandas as pd
from linearmodels.panel import RandomEffects
import numpy as np
import statsmodels.api as sm
from sklearn.preprocessing import StandardScaler

# Step 3: Mount Google Drive (if using Google Drive for data storage)
from google.colab import drive
drive.mount('/content/drive')

# Step 4: Load the data
file_path = '/content/drive/My Drive/panel_data_2000_2024.csv'  # Adjust the path accordingly
data = pd.read_csv(file_path)

# Step 5: Prepare panel data
data['panel_id'] = data['stateid'].astype(str) + '_' + data['districtid'].astype(str)
data.set_index(['panel_id', 'year'], inplace=True)

# Step 6: Filter the data for the sample period (2000 to 2020)
data_sample = data.loc[data.index.get_level_values('year') <= 2020]

# Step 7: Calculate 1-year lag for the average price
data_sample['avg_price_lag1'] = data_sample[['price_crop10m11', 'price_crop11m11', 'price_crop12m11']].mean(axis=1).groupby(level='panel_id').shift(1)

# Step 8: Generate variables dividing msp11 and the lagged avg_price by fert1, fert2, and fert3
for fert in ['fert1', 'fert2', 'fert3']:
    data_sample[f'msp11_{fert}'] = data_sample['msp11'] / data_sample[fert]
    data_sample[f'avg_price_{fert}'] = data_sample['avg_price_lag1'] / data_sample[fert]

# Step 9: Prepare variables for analysis
rainfall_vars = ['rainfall5', 'rainfall6', 'rainfall7', 'rainfall8', 'rainfall9', 'rainfall10', 'rainfall11', 'rainfall12']
reservoir_vars = ['res_1', 'res_2', 'res_3', 'res_4', 'res_5', 'res_6', 'res_7', 'res_8', 'res_9']
temperature_vars = ['july1_max', 'aug1_max', 'sep1_max', 'oct1_max', 'aug2_max', 'sep2_max', 'oct2_max', 'july1_min', 'aug1_min', 'sep1_min', 'oct1_min', 'july2_min', 'aug2_min', 'sep2_min', 'oct2_min']

# Step 10: Convert soil_type into dummy variables
soil_dummies = pd.get_dummies(data_sample['soil_type'], prefix='soil', drop_first=True)
data_sample = pd.concat([data_sample, soil_dummies], axis=1)

# Step 11: Generate interaction terms
for irrigation_type in ['irrigation_nia', 'irrigation_canal', 'irrigation_well', 'irrigation_tank', 'irrigation_others']:
    for var in rainfall_vars + reservoir_vars:
        data_sample[f'{var}_{irrigation_type}'] = data_sample[var] * data_sample[irrigation_type]
    data_sample[f'groundwater_{irrigation_type}'] = data_sample['groundwater'] * data_sample[irrigation_type]

# Step 12: Define dependent variable 'y'
y = data_sample['yield'].loc[data_sample.index]

# Step 13: Function to check significance and aggregate variables
def check_significance_and_average(y, data_sample, variables):
    significant_vars = []
    for var in variables:
        # Create a model with the current variable included
        X_temp = data_sample[significant_vars + [var]].dropna()
        y_temp = y.loc[X_temp.index]

        if not y_temp.empty and not X_temp.empty:
            temp_model = RandomEffects(y_temp, X_temp)
            results = temp_model.fit()

            # Check if significant at 5% level
            if results.pvalues[var] < 0.05:
                significant_vars.append(var)
    return significant_vars

# Step 14: Analyze each group of variables
# For price-related variables, only keep positive significant ones
price_vars = ['msp11_fert1', 'msp11_fert2', 'msp11_fert3', 'avg_price_fert1', 'avg_price_fert2', 'avg_price_fert3']
significant_price_vars = check_significance_and_average(y, data_sample, price_vars)

# Calculate average for significant price variables
if significant_price_vars:
    data_sample['average_price'] = data_sample[significant_price_vars].mean(axis=1)
else:
    data_sample['average_price'] = 0

# For groundwater variables
groundwater_vars = ['groundwater']
significant_groundwater_vars = check_significance_and_average(y, data_sample, groundwater_vars)

if significant_groundwater_vars:
    data_sample['average_groundwater'] = data_sample[significant_groundwater_vars].mean(axis=1)
else:
    data_sample['average_groundwater'] = 0

# For reservoir variables, sum them
data_sample['total_reservoir'] = data_sample[reservoir_vars].sum(axis=1)

# For rainfall variables, average them
data_sample['average_rainfall'] = data_sample[rainfall_vars].mean(axis=1)

# For irrigation variables, average them
irrigation_vars = ['irrigation_nia', 'irrigation_canal', 'irrigation_well', 'irrigation_tank', 'irrigation_others']
data_sample['average_irrigation'] = data_sample[irrigation_vars].mean(axis=1)

# For temperature variables, average them
data_sample['average_temperature'] = data_sample[temperature_vars].mean(axis=1)

# Step 15: Create a DataFrame with selected significant variables
selected_vars = ['average_price', 'average_groundwater', 'total_reservoir', 'average_rainfall', 'average_irrigation', 'average_temperature']

# Ensure y and X_final have the same number of rows after missing data handling
X_final = data_sample[selected_vars].dropna()
y_final = y.loc[X_final.index]  # Align y with X_final

# Step 16: Run the final Random Effects Model
X_final = sm.add_constant(X_final)  # Add constant term to the model
re_model_final = RandomEffects(y_final, X_final)
re_results_final = re_model_final.fit()

# Print the final model summary
print(re_results_final.summary)

# Extract and print the constant term
if 'const' in re_results_final.params:
    print("Constant term (Intercept):", re_results_final.params['const'])
else:
    print("Constant term not included in the model.")

# Step 17: Forecast yield for 2021 to 2024 using significant positive variables
# Filter data for the forecast period
forecast_data = data.loc[(data.index.get_level_values('year') >= 2021) & (data.index.get_level_values('year') <= 2024)]

# Ensure to reset the index to keep 'panel_id' and 'year' as columns
forecast_data.reset_index(inplace=True)

# Step 18: Create average and total variables for forecasting
forecast_data['average_price'] = data_sample['average_price'].mean()
forecast_data['average_groundwater'] = data_sample['average_groundwater'].mean()
forecast_data['total_reservoir'] = data_sample['total_reservoir'].mean()
forecast_data['average_rainfall'] = data_sample['average_rainfall'].mean()
forecast_data['average_irrigation'] = data_sample['average_irrigation'].mean()
forecast_data['average_temperature'] = data_sample['average_temperature'].mean()

# Create the MultiIndex for forecasting
forecast_data.set_index(['panel_id', 'year'], inplace=True)

# Prepare final input for prediction
X_forecast = forecast_data[['average_price', 'average_groundwater', 'total_reservoir', 'average_rainfall', 'average_irrigation', 'average_temperature']]
X_forecast = sm.add_constant(X_forecast)  # Add constant term to forecast data

# Step 19: Predict yields for 2021 to 2024
forecast_data['predicted_yield'] = re_results_final.predict(X_forecast)

# Step 20: Display forecasted results
print(forecast_data[['year', 'predicted_yield']])


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


<ipython-input-17-5a3e8d2d12fd>:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_sample['avg_price_lag1'] = data_sample[['price_crop10m11', 'price_crop11m11', 'price_crop12m11']].mean(axis=1).groupby(level='panel_id').shift(1)
<ipython-input-17-5a3e8d2d12fd>:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_sample[f'msp11_{fert}'] = data_sample['msp11'] / data_sample[fert]
<ipython-input-17-5a3e8d2d12fd>:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFram

                        RandomEffects Estimation Summary                        
Dep. Variable:                  yield   R-squared:                        0.4532
Estimator:              RandomEffects   R-squared (Between):             -0.2634
No. Observations:                1287   R-squared (Within):               0.4937
Date:                Wed, Dec 11 2024   R-squared (Overall):              0.1511
Time:                        06:53:49   Log-likelihood                   -695.31
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      176.82
Entities:                          62   P-value                           0.0000
Avg Obs:                       20.758   Distribution:                  F(6,1280)
Min Obs:                       13.000                                           
Max Obs:                       21.000   F-statistic (robust):             176.72
                            

/usr/local/lib/python3.10/dist-packages/linearmodels/panel/model.py:2759: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  super().__init__(dependent, exog, weights=weights, check_rank=check_rank)
<ipython-input-17-5a3e8d2d12fd>:131: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  forecast_data['average_price'] = data_sample['average_price'].mean()
<ipython-input-17-5a3e8d2d12fd>:132: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  forecast_data['average_groundwate

ValueError: exog does not have the correct number of columns. Saw 6, expected
7. This can happen since exog is converted to a PanelData object
before computing the fitted value. The best practice is to pass a DataFrame
with a 2-level MultiIndex containing the entity- and time-ids.

In [18]:
# Step 1: Install necessary packages
!pip install linearmodels statsmodels

# Step 2: Import libraries
import pandas as pd
from linearmodels.panel import RandomEffects
import statsmodels.api as sm

# Step 3: Mount Google Drive (if using Google Drive for data storage)
from google.colab import drive
drive.mount('/content/drive')

# Step 4: Load the data
file_path = '/content/drive/My Drive/panel_data_2000_2024.csv'  # Adjust the path accordingly
data = pd.read_csv(file_path)

# Step 5: Prepare panel data
data['panel_id'] = data['stateid'].astype(str) + '_' + data['districtid'].astype(str)
data.set_index(['panel_id', 'year'], inplace=True)

# Step 6: Filter the data for the sample period (2000 to 2020)
data_sample = data.loc[data.index.get_level_values('year') <= 2020]

# Step 7: Calculate 1-year lag for the average price
data_sample['avg_price_lag1'] = data_sample[['price_crop10m11', 'price_crop11m11', 'price_crop12m11']].mean(axis=1).groupby(level='panel_id').shift(1)

# Step 8: Generate variables dividing msp11 and the lagged avg_price by fert1, fert2, and fert3
for fert in ['fert1', 'fert2', 'fert3']:
    data_sample[f'msp11_{fert}'] = data_sample['msp11'] / data_sample[fert]
    data_sample[f'avg_price_{fert}'] = data_sample['avg_price_lag1'] / data_sample[fert]

# Step 9: Prepare variables for analysis
rainfall_vars = ['rainfall5', 'rainfall6', 'rainfall7', 'rainfall8', 'rainfall9', 'rainfall10', 'rainfall11', 'rainfall12']
reservoir_vars = ['res_1', 'res_2', 'res_3', 'res_4', 'res_5', 'res_6', 'res_7', 'res_8', 'res_9']
temperature_vars = ['july1_max', 'aug1_max', 'sep1_max', 'oct1_max', 'aug2_max', 'sep2_max', 'oct2_max', 'july1_min', 'aug1_min', 'sep1_min', 'oct1_min', 'july2_min', 'aug2_min', 'sep2_min', 'oct2_min']

# Step 10: Convert soil_type into dummy variables
soil_dummies = pd.get_dummies(data_sample['soil_type'], prefix='soil', drop_first=True)
data_sample = pd.concat([data_sample, soil_dummies], axis=1)

# Step 11: Generate interaction terms
for irrigation_type in ['irrigation_nia', 'irrigation_canal', 'irrigation_well', 'irrigation_tank', 'irrigation_others']:
    for var in rainfall_vars + reservoir_vars:
        data_sample[f'{var}_{irrigation_type}'] = data_sample[var] * data_sample[irrigation_type]
    data_sample[f'groundwater_{irrigation_type}'] = data_sample['groundwater'] * data_sample[irrigation_type]

# Step 12: Define dependent variable 'y'
y = data_sample['yield']

# Step 13: Prepare the independent variables for the final model
# Average variables
data_sample['average_price'] = data_sample[['msp11_fert1', 'msp11_fert2', 'msp11_fert3', 'avg_price_fert1', 'avg_price_fert2', 'avg_price_fert3']].mean(axis=1)
data_sample['average_groundwater'] = data_sample[['groundwater']].mean(axis=1)
data_sample['total_reservoir'] = data_sample[reservoir_vars].sum(axis=1)
data_sample['average_rainfall'] = data_sample[rainfall_vars].mean(axis=1)
data_sample['average_irrigation'] = data_sample[['irrigation_nia', 'irrigation_canal', 'irrigation_well', 'irrigation_tank', 'irrigation_others']].mean(axis=1)
data_sample['average_temperature'] = data_sample[temperature_vars].mean(axis=1)

# Step 14: Define the final independent variables and add the constant
X_final = data_sample[['average_price', 'average_groundwater', 'total_reservoir', 'average_rainfall', 'average_irrigation', 'average_temperature']]
X_final = sm.add_constant(X_final)  # Adding constant (intercept) term

# Step 15: Run the final Random Effects Model
re_model_final = RandomEffects(y, X_final)
re_results_final = re_model_final.fit()

# Print the summary of the Random Effects model
print(re_results_final.summary)

# Step 16: Forecast yield for 2021 to 2024
forecast_data = data.loc[(data.index.get_level_values('year') >= 2021) & (data.index.get_level_values('year') <= 2024)]
forecast_data.reset_index(inplace=True)  # Resetting index to include 'panel_id' and 'year' as columns

# Step 17: Create forecast variables (average and total)
forecast_data['average_price'] = data_sample['average_price'].mean()
forecast_data['average_groundwater'] = data_sample['average_groundwater'].mean()
forecast_data['total_reservoir'] = data_sample['total_reservoir'].mean()
forecast_data['average_rainfall'] = data_sample['average_rainfall'].mean()
forecast_data['average_irrigation'] = data_sample['average_irrigation'].mean()
forecast_data['average_temperature'] = data_sample['average_temperature'].mean()

# Step 18: Create the MultiIndex for forecasting (must match the training data)
forecast_data.set_index(['panel_id', 'year'], inplace=True)

# Step 19: Prepare the forecast data for prediction by selecting relevant columns
X_forecast = forecast_data[['average_price', 'average_groundwater', 'total_reservoir',
                            'average_rainfall', 'average_irrigation', 'average_temperature']]
X_forecast = sm.add_constant(X_forecast)  # Add constant (intercept) term

# Step 20: Predict yields for 2021 to 2024 using the fitted model
forecast_data['predicted_yield'] = re_results_final.predict(X_forecast)

# Step 21: Display the forecasted results
forecast_results = forecast_data[['panel_id', 'year', 'predicted_yield']]

# Print forecasted results
print(forecast_results)

# Optional: Save forecasted results to a CSV file
forecast_results.to_csv("/content/drive/My Drive/forecasted_yields_2021_2024.csv", index=False)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
                        RandomEffects Estimation Summary                        
Dep. Variable:                  yield   R-squared:                        0.4532
Estimator:              RandomEffects   R-squared (Between):             -0.2634
No. Observations:                1287   R-squared (Within):               0.4937
Date:                Wed, Dec 11 2024   R-squared (Overall):              0.1511
Time:                        06:56:11   Log-likelihood                   -695.31
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      176.82
Entities:                          62   P-value                           0.0000
Avg Obs:                       20.758   Distribution:                  F(6,1280)
Min Obs:                       13.000                        

<ipython-input-18-4991b5abb4fa>:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_sample['avg_price_lag1'] = data_sample[['price_crop10m11', 'price_crop11m11', 'price_crop12m11']].mean(axis=1).groupby(level='panel_id').shift(1)
<ipython-input-18-4991b5abb4fa>:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_sample[f'msp11_{fert}'] = data_sample['msp11'] / data_sample[fert]
<ipython-input-18-4991b5abb4fa>:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFram

ValueError: exog does not have the correct number of columns. Saw 6, expected
7. This can happen since exog is converted to a PanelData object
before computing the fitted value. The best practice is to pass a DataFrame
with a 2-level MultiIndex containing the entity- and time-ids.

In [1]:
# Step 1: Install necessary packages
!pip install linearmodels statsmodels

# Step 2: Import libraries
import pandas as pd
from linearmodels.panel import RandomEffects
import statsmodels.api as sm

# Step 3: Mount Google Drive (if using Google Drive for data storage)
from google.colab import drive
drive.mount('/content/drive')

# Step 4: Load the data
file_path = '/content/drive/My Drive/panel_data_2000_2024_1.csv'  # Adjust the path accordingly
data = pd.read_csv(file_path)

# Step 5: Prepare panel data
data['panel_id'] = data['stateid'].astype(str) + '_' + data['districtid'].astype(str)
data.set_index(['panel_id', 'year'], inplace=True)

# Step 6: Filter the data for the sample period (2000 to 2020)
data_sample = data.loc[data.index.get_level_values('year') <= 2020]

# Step 7: Calculate 1-year lag for the average price
data_sample['avg_price_lag1'] = data_sample[['price_crop10m11', 'price_crop11m11', 'price_crop12m11']].mean(axis=1).groupby(level='panel_id').shift(1)

# Step 8: Generate variables dividing msp11 and the lagged avg_price by fert1, fert2, and fert3
for fert in ['fert1', 'fert2', 'fert3']:
    data_sample[f'msp11_{fert}'] = data_sample['msp11'] / data_sample[fert]
    data_sample[f'avg_price_{fert}'] = data_sample['avg_price_lag1'] / data_sample[fert]

# Step 9: Prepare variables for analysis
rainfall_vars = ['rainfall5', 'rainfall6', 'rainfall7', 'rainfall8', 'rainfall9', 'rainfall10', 'rainfall11', 'rainfall12']
reservoir_vars = ['res_1', 'res_2', 'res_3', 'res_4', 'res_5', 'res_6', 'res_7', 'res_8', 'res_9']
temperature_vars = ['july1_max', 'aug1_max', 'sep1_max', 'oct1_max', 'aug2_max', 'sep2_max', 'oct2_max', 'july1_min', 'aug1_min', 'sep1_min', 'oct1_min', 'july2_min', 'aug2_min', 'sep2_min', 'oct2_min']

# Step 10: Convert soil_type into dummy variables
soil_dummies = pd.get_dummies(data_sample['soil_type'], prefix='soil', drop_first=True)
data_sample = pd.concat([data_sample, soil_dummies], axis=1)

# Step 11: Generate interaction terms
for irrigation_type in ['irrigation_nia', 'irrigation_canal', 'irrigation_well', 'irrigation_tank', 'irrigation_others']:
    for var in rainfall_vars + reservoir_vars:
        data_sample[f'{var}_{irrigation_type}'] = data_sample[var] * data_sample[irrigation_type]
    data_sample[f'groundwater_{irrigation_type}'] = data_sample['groundwater'] * data_sample[irrigation_type]

# Step 12: Define dependent variable 'y'
y = data_sample['yield']

# Step 13: Prepare the independent variables for the final model
# Average variables
data_sample['average_price'] = data_sample[['msp11_fert1', 'msp11_fert2', 'msp11_fert3', 'avg_price_fert1', 'avg_price_fert2', 'avg_price_fert3']].mean(axis=1)
data_sample['average_groundwater'] = data_sample[['groundwater']].mean(axis=1)
data_sample['total_reservoir'] = data_sample[reservoir_vars].sum(axis=1)
data_sample['average_rainfall'] = data_sample[rainfall_vars].mean(axis=1)
data_sample['average_irrigation'] = data_sample[['irrigation_nia', 'irrigation_canal', 'irrigation_well', 'irrigation_tank', 'irrigation_others']].mean(axis=1)
data_sample['average_temperature'] = data_sample[temperature_vars].mean(axis=1)

# Step 14: Define the final independent variables and add the constant
X_final = data_sample[['average_price', 'average_groundwater', 'total_reservoir', 'average_rainfall', 'average_irrigation', 'average_temperature']]
X_final = sm.add_constant(X_final)  # Adding constant (intercept) term

# Step 15: Run the final Random Effects Model
re_model_final = RandomEffects(y, X_final)
re_results_final = re_model_final.fit()

# Print the summary of the Random Effects model
print(re_results_final.summary)

# Step 16: Forecast yield for 2021 to 2024
forecast_data = data.loc[(data.index.get_level_values('year') >= 2021) & (data.index.get_level_values('year') <= 2024)]
forecast_data.reset_index(inplace=True)  # Resetting index to include 'panel_id' and 'year' as columns

# Step 17: Create forecast variables (average and total)
forecast_data['average_price'] = data_sample['average_price'].mean()
forecast_data['average_groundwater'] = data_sample['average_groundwater'].mean()
forecast_data['total_reservoir'] = data_sample['total_reservoir'].mean()
forecast_data['average_rainfall'] = data_sample['average_rainfall'].mean()
forecast_data['average_irrigation'] = data_sample['average_irrigation'].mean()
forecast_data['average_temperature'] = data_sample['average_temperature'].mean()

# Step 18: Create the MultiIndex for forecasting (must match the training data)
forecast_data.set_index(['panel_id', 'year'], inplace=True)

# Step 19: Prepare the forecast data for prediction by selecting relevant columns
X_forecast = forecast_data[['average_price', 'average_groundwater', 'total_reservoir',
                            'average_rainfall', 'average_irrigation', 'average_temperature']]

# Step 20: Add constant (intercept) term to forecast data
X_forecast = sm.add_constant(X_forecast)  # Adding constant (intercept) term

# Step 21: Check for the correct number of columns in X_forecast
print(f"X_forecast shape: {X_forecast.shape}")
print(f"Expected number of columns: {X_final.shape[1]}")

# Step 22: Predict yields for 2021 to 2024 using the fitted model
forecast_data['predicted_yield'] = re_results_final.predict(X_forecast)

# Step 23: Display the forecasted results
forecast_results = forecast_data[['panel_id', 'year', 'predicted_yield']]

# Print forecasted results
print(forecast_results)

# Optional: Save forecasted results to a CSV file
forecast_results.to_csv("/content/drive/My Drive/forecasted_yields_2021_2024.csv", index=False)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 52.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.5/94.5 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 2.8 MB/s eta 0:00:00
Mounted at /content/drive


<ipython-input-1-1506915e5cbe>:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_sample['avg_price_lag1'] = data_sample[['price_crop10m11', 'price_crop11m11', 'price_crop12m11']].mean(axis=1).groupby(level='panel_id').shift(1)
<ipython-input-1-1506915e5cbe>:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_sample[f'msp11_{fert}'] = data_sample['msp11'] / data_sample[fert]
<ipython-input-1-1506915e5cbe>:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.


                        RandomEffects Estimation Summary                        
Dep. Variable:                  yield   R-squared:                        0.4598
Estimator:              RandomEffects   R-squared (Between):             -0.1938
No. Observations:                1287   R-squared (Within):               0.4947
Date:                Wed, Dec 11 2024   R-squared (Overall):              0.1829
Time:                        09:59:37   Log-likelihood                   -685.47
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      181.61
Entities:                          62   P-value                           0.0000
Avg Obs:                       20.758   Distribution:                  F(6,1280)
Min Obs:                       13.000                                           
Max Obs:                       21.000   F-statistic (robust):             181.51
                            

<ipython-input-1-1506915e5cbe>:80: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  forecast_data['average_temperature'] = data_sample['average_temperature'].mean()


ValueError: exog does not have the correct number of columns. Saw 6, expected
7. This can happen since exog is converted to a PanelData object
before computing the fitted value. The best practice is to pass a DataFrame
with a 2-level MultiIndex containing the entity- and time-ids.